In [1]:
:set -XOverloadedStrings

import Control.Monad (unless)
import Control.Monad.Trans.State.Strict as SS
import Data.Attoparsec.ByteString
import Data.Byteable
import qualified Data.Map.Strict        as M
import qualified Data.ByteString.Lazy   as L
import qualified Data.ByteString        as B
import qualified Codec.Compression.Zlib as Z
import Pipes
import Prelude hiding (take)
import qualified Pipes.ByteString as PB
import qualified Pipes.Prelude as PP
import qualified Pipes.Zlib as PZ
import qualified Pipes.Attoparsec as PA
import qualified System.IO as SI

import Duffer.Pack
import Duffer.Pack.Parser
import Duffer.Pack.Entries
import Duffer.Pack.Streaming

In [2]:
compressedContent = L.toStrict $ Z.compress "compressed content"
uncompressedContent = L.toStrict $ "uncompressed content"
mingled = B.concat [compressedContent, uncompressedContent, compressedContent]
mingledProducer = yield mingled
decompressed = PZ.decompress' PZ.defaultWindowBits mingledProducer
(Right (value, restProducer)) <- next decompressed
value
(Left (Left finalProducer)) <- next restProducer
(Right (final, _)) <- next finalProducer
final

"compressed content"

"uncompressed contentx\156K\206\207-(J-.NMQH\206\207+I\205+\SOH\NULE\179\aQ"

In [5]:
import qualified Prelude as P

handle  <- SI.openFile ".git/objects/pack/pack-23a94077d4edc73dd898189c8fa17a49c5e3ad23.pack" SI.ReadMode
handleP = PB.fromHandle handle
((start, count), entries) <- runStateT parsePackfileStart handleP
result <- loopEntries entries start 14 M.empty
rest = snd result
:t rest
(header, decompressedP, level) <- evalStateT getNextEntry rest
level
:t decompressedP
Right (v, p) <- next decompressedP
v
streamedMap = fst result
M.keys $ fst result
iBSMap <- indexedByteStringMap ".git/objects/pack/pack-23a94077d4edc73dd898189c8fa17a49c5e3ad23.idx"
P.take 15 $ M.keys iBSMap
(streamedMap M.! 2285) == (iBSMap M.! 2285)

rest :: Producer ByteString IO ()

40

decompressedP :: Producer ByteString IO (Either (Producer ByteString IO ()) ())

ZlibException (-3)